In [5]:
import pandas as pd
import numpy as np
from datetime import datetime

In [6]:
# Load the raw data
netflix_df = pd.read_csv('/Users/vanessaokosun/Downloads/netflix_titles.csv')

## 1. Handeling Missing Values

In [7]:
# Check missing values
missing_values = netflix_df.isnull().sum()
print("Missing values by column:")
print(missing_values[missing_values > 0])

# Handle missing values appropriately
netflix_df['director'] = netflix_df['director'].fillna('Unknown')
netflix_df['cast'] = netflix_df['cast'].fillna('Not Available')
netflix_df['country'] = netflix_df['country'].fillna('Unknown')
netflix_df['rating'] = netflix_df['rating'].fillna('Not Rated')
netflix_df['date_added'] = pd.to_datetime(netflix_df['date_added'], errors='coerce')


Missing values by column:
director      2634
cast           825
country        831
date_added      10
rating           4
duration         3
dtype: int64


## 2. FEATURE ENGINEERING

In [25]:
# Extract year and month added
netflix_df['year_added'] = netflix_df['date_added'].dt.year
netflix_df['month_added'] = netflix_df['date_added'].dt.month
netflix_df['day_added'] = netflix_df['date_added'].dt.day

# Extract content age (years since release)
netflix_df['release_year'] = pd.to_datetime(netflix_df['release_year'], errors='coerce')
netflix_df['content_age'] = (datetime.now() - netflix_df['release_year']).dt.days // 365
netflix_df['content_age'] = netflix_df['content_age'].fillna(0).astype(int)

# Create duration_minutes for movies
def extract_duration(row):
    if isinstance(row, str):
        if 'Season' in row:
            return 0
        else:
            return int(row.split()[0])
    return 0
netflix_df['duration_minutes'] = netflix_df['duration'].apply(extract_duration)

# Convert duration to minutes for movies and number of seasons for TV shows
# Ensure 'duration' is string for string operations, handle NaN safely
duration_str = netflix_df['duration'].fillna('').astype(str)

netflix_df['duration_minutes'] = np.where(
    duration_str.str.contains('Season', na=False),
    duration_str.str.extract('(\d+)')[0].fillna(0).astype(int),
    duration_str.str.extract('(\d+)')[0].fillna(0).astype(int)
)

# Extract main genre (first listed genre)
netflix_df['main_genre'] = netflix_df['listed_in'].str.split(',').str[0].str.strip()

# Create genre flags for top genres
top_genres = ['Action & Adventure', 'Children & Family Movies', 'Comedies', 'Documentaries', 'Dramas', 'Horror Movies', 'International Movies', 'Romantic Movies', 'Sci-Fi & Fantasy', 'Thrillers']
for genre in top_genres:
    netflix_df[genre] = np.where(netflix_df['listed_in'].str.contains(genre), 1, 0)
    
# Create flag for potential Netflix Originals (simplified approach)
if 'production_company' in netflix_df.columns:
    netflix_df['is_original'] = np.where(netflix_df['production_company'].str.contains('Netflix', na=False), 1, 0)
else:
    netflix_df['is_original'] = 0  # or np.nan if you prefer

# Create flag for content added in the last 30 days
netflix_df['recently_added'] = np.where((datetime.now() - netflix_df['date_added']).dt.days <= 30, 1, 0)

# Create flag for content added in the last 90 days
netflix_df['recently_added_90'] = np.where((datetime.now() - netflix_df['date_added']).dt.days <= 90, 1, 0)

# Create flag for content added in the last year
netflix_df['recently_added_year'] = np.where((datetime.now() - netflix_df['date_added']).dt.days <= 365, 1, 0)



<>:27: SyntaxWarning: invalid escape sequence '\d'
<>:28: SyntaxWarning: invalid escape sequence '\d'
<>:27: SyntaxWarning: invalid escape sequence '\d'
<>:28: SyntaxWarning: invalid escape sequence '\d'
/var/folders/l3/8dwhb5_n1zq1zskmr1h73rx80000gn/T/ipykernel_24447/2720114345.py:27: SyntaxWarning: invalid escape sequence '\d'
  duration_str.str.extract('(\d+)')[0].fillna(0).astype(int),
/var/folders/l3/8dwhb5_n1zq1zskmr1h73rx80000gn/T/ipykernel_24447/2720114345.py:28: SyntaxWarning: invalid escape sequence '\d'
  duration_str.str.extract('(\d+)')[0].fillna(0).astype(int)
/var/folders/l3/8dwhb5_n1zq1zskmr1h73rx80000gn/T/ipykernel_24447/2720114345.py:27: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  duration_str.str.extract('(\d+)')[0].fillna(0).astype(int),
/var/folde

## 3. Data Validation & Consistency 

In [26]:
# Check for duplicated show_id values
duplicate_ids = netflix_df['show_id'].duplicated().sum()
print(f"Duplicate IDs: {duplicate_ids}")

# Remove any duplicates if they exist
if duplicate_ids > 0:
    netflix_df = netflix_df.drop_duplicates(subset=['show_id'])

# Standardize country names (focusing on main production countries)
country_mapping = {
    'United States': ['US', 'USA', 'U.S.', 'U.S.A.', 'United States of America'],
    'United Kingdom': ['UK', 'U.K.', 'Britain', 'Great Britain'],
    # Add more mappings as needed
}

for standard, variants in country_mapping.items():
    for variant in variants:
        netflix_df['country'] = netflix_df['country'].str.replace(variant, standard, regex=False)

Duplicate IDs: 0


## 4. Saved Clean Dataset

In [31]:
netflix_df.to_csv('/Users/vanessaokosun/streaming-wars-bi-dashboard/streaming-wars-bi-dashboard/data/cleaned_netflix_titles.csv', index=False)

print("Data cleaning complete. Cleaned file saved")

Data cleaning complete. Cleaned file saved
